# Ins and Outs of Polynomials

This document serves as a reference for how prysm is set up to work with polynomials, in the context of OPD or surface figure error.  Much of what differentiates prysm's API in this area has to do with the fact that it [expects the grid to exist at the user level](./how-prysm-works.ipynb#Grids), but there are some deep and consequential implementation differences, too.  Before we get into those, we will create a working grid and a mask for visualization:

In [ ]:
import numpy as np

from prysm.coordinates import make_xy_grid, cart_to_polar
from prysm.geometry import circle

from matplotlib import pyplot as plt

x, y = make_xy_grid(256, diameter=2)
r, t = cart_to_polar(x, y)
mask = ~circle(1,r)  # invert: mask is true outside the circle

## Table of Contents

### Bases:

- [Hopkins](#Hopkins)
- [Zernike](#Zernike)
- [Jacobi](#Jacobi)
- [Chebyshev](#Chebyshev)
- [Legendre](#Legendre)
- [Dickson](#Dickson)
- [Qs](#Qs)

Note that all polynomial types allow evaluation for arbitrary order.  First partial derivatives can be computed using the format `{polynomial}_der` or `{polynomial}_der_seq`.  1D polynomials are differentiated with respect to x.  2D polynomials are differentiated with respect to the coordiates they are defined over, e.g. rho, theta for Zernike and Q-type polynomials.  Differentiation is done analytically and does not rely on finite differences.

### Fitting and Non-Circular Domains

- [Fitting](#Fitting)
- [Annular Domains](#Annular-Domains)
- [Arbitrary Domains](#Arbitrary-Domains)

## Hopkins

The simplest polynomials are Hopkins':

$$ \text{OPD} = W_{abc} \left[\cos\left(a\cdot\theta\right) \cdot \rho^b \cdot H^c \right]$$

for some set of coefficients.  The usage of this should not be surprising, for $W_{131}$, coma one can write:

In [ ]:
from prysm.polynomials import hopkins
cma = hopkins(1, 3, 1, r, t, 1)
cma[mask]=np.nan
plt.imshow(cma)
ax = plt.gca()

Note that we defined our grid to have a radius of 1, but often you may hold two copies of r, one which is normalized by some reference radius for polynomial evaluation, and one which is not for pupil geometry evaluation.  There is no further complexity in using Hopkins' polynomials.

## Zernike

prysm has a fairly granular implementation of Zernike polynomials, and expects its users to assemble the pieces to synthesize higher order functionality.  The basic building block is the `zernike_nm` function, which takes azimuthal and radial orders n and m, as in $Z_n^m$.  For example, to compute the equivalent "primary coma" Zernike mode as the hopkins example, one would:

In [ ]:
from prysm.polynomials import zernike_nm
cmaZ = zernike_nm(3,1, r,t, norm=True)
cmaZ[mask]=np.nan
plt.imshow(cmaZ)

Note that the terms can be orthonormalized (given unit RMS) or not, based on the `norm` kwarg.  The order `m` can be negative to give access to the sinusoidal terms instead of cosinusoidal.  If you wish to work with a particular ordering scheme, prysm supports Fringe, Noll, and ANSI out of the box, all of which start counting at 1.

In [ ]:
from prysm.polynomials import noll_to_nm, fringe_to_nm, ansi_j_to_nm

n, m = fringe_to_nm(9)
sphZ = zernike_nm(n, m, r, t, norm=False)
sphZ[mask]=np.nan
plt.imshow(sphZ)

These functions are not iterator-aware and should be used with, say, a list comprehension.

If you wish to compute Zernikes much more quickly, the underlying implementation in prysm allows the work in computing lower order terms to be used to compute the higher order terms.  The Zernike polynomials are fundamentally two "pieces" which get multiplied.  The radial basis is where much of the work lives, and most programs that do not type out closed form solutions use Rodrigues' technique to compute the radial basis:

$$
R_n^m (\rho) = \sum_{k=0}^{\frac{n-m}{2}} \frac{(-1)^k (n-k)!}{k!(\frac{n+m}{2}-k)!(\frac{n-m}{2}-k)!}\rho^{n-2k} \tag{1}
$$

prysm does not do this, and instead uses the fact that the radial polynomial is a Jacobi polynomial under a change-of-basis:

$$
R_n^m (\rho) = P_\frac{n-m}{2}^{\left(0,|m|\right)}\left(2\rho^2 - 1\right) \tag{2}
$$

And the jacobi polynomials can be computed using a recurrence relation:
$$
a \cdot P_n^{(\alpha,\beta)} = b \cdot x \cdot P_{n-1}^{(\alpha,\beta)} - c \cdot P_{n-2}^{(\alpha,\beta)} \tag{3}
$$

In other words, for a given $m$, you can compute $R$ for $n=3$ from $R$ for $n=2$ and $n=1$, and so on until you reach the highest value of N.  Because the sum in the Rodrigues formulation is increasingly large as $n,m$ grow, it has worse than linear time complexity.  Because the recurrrence in Eq. (3) does not change as $n,m$ grow it _does_ have linear time complexity.

The use of this recurrence relation is hidden from the user in the `zernike_nm` function, and the recurrence relation is for a so-called auxiliary polynomial ($R$), so the Zernike polynomials themselves are not useful for this recurrence.  You _can_ make use of it by calling the `zernike_nm_seq` function, a naming that will become familiar by the end of this reference guide.  Consider the first 16 Fringe Zernikes:

In [ ]:
from prysm.polynomials import zernike_nm_seq

nms = [fringe_to_nm(i) for i in range(1,36)]

# zernike_nm_seq returns a cube of shape (len(nms), *r.shape)
%timeit basis = zernike_nm_seq(nms, r, t) # implicit norm=True

Compare the timing to not using the seq flavored version:

In [ ]:
%%timeit
for n, m in nms:
    zernike_nm(n, m, r, t)

These is no benefit other than performance to the `_seq` version of the function, but their usage is highly encouraged.  A side benefit to the recurrence relation is that it is numerically stable to higher order than Rodrigues' expression, so you can compute higher order Zernike polynomials without numerical errors.  This is an especially useful property for using lower-precision data types like float32, since they will suffer from numerical imprecision earlier.

## Jacobi 
Of course, because the Zernike polynomials are related to them you also have access to the Jacobi polynomials:

In [ ]:
from prysm.polynomials import jacobi, jacobi_seq

x_ = x[0,:] # not required to be 1D, just for example
plt.plot(x_, jacobi_seq([1,2,3,4,5],0,0,x_).T)

These shapes may be familiar to Zernike polynomials.

## Chebyshev

All four types of Chevyshev polynomials are supported.  They are just special cases of Jacobi polynomials.  The first and second kind are common:

$$ T(x) = \text{cheby1}  \equiv P_n^{\left(-0.5,-0.5\right)}(x) \quad / \quad P_n^{\left(-0.5,-0.5\right)}(1)$$
$$ U(x) = \text{cheby2}  \equiv (n+1) P_n^{\left(0.5,0.5\right)}(x) \quad / \quad P_n^{\left(0.5,0.5\right)}(1)$$

While the third and fourth kind are more obscure:

$$ V(x) = \text{cheby3}  \equiv P_n^{\left(-0.5,0.5\right)}(x) \quad / \quad P_n^{\left(-0.5,0.5\right)}(1)$$
$$ W(x) = \text{cheby4}  \equiv (2n+1) P_n^{\left(0.5,-0.5\right)}(x) \quad / \quad P_n^{\left(0.5,-0.5\right)}(1)$$

In [ ]:
from prysm.polynomials import cheby1, cheby2, cheby1_seq, cheby3, cheby4

In [ ]:
fs = [cheby1, cheby2, cheby3, cheby4]
n = 5
for f in fs:
    plt.plot(x_, f(n,x_))
plt.legend(['first kind', 'second kind', 'third kind', 'fourth kind'])

## Legendre

These polynomials are just a special case of Jacobi polynomials:

$$ \text{legendre} \equiv P_n^{\left(0,0\right)}(x) $$

Usage follows from the [Chebyshev](#Chebyshev) exactly, except the functions are prefixed by `legendre`.  No plots here - they would be identical to those from the Jacobi section.

In [ ]:
from prysm.polynomials import legendre, legendre_seq

## Dickson
These polynomials use a two-term recurrence relation, but are not based on Jacobi polynomials.  For the Dickson polynomials of the first kind $D$:

$$
\begin{align}
D_{n+1} &= x \cdot D_n - \alpha D_{n-1} \\
D_0 &= 2 \\
D_1 &= x
\end{align}
$$

And the second kind $E$:

$$
\begin{align}
E_{n+1} &= x \cdot E_n - \alpha E_{n-1} \\
E_0 &= 1 \\
E_1 &= x
\end{align}
$$

The interface is once again the same:

In [ ]:
from prysm.polynomials import (
    dickson1, dickson1_seq,
    dickson2, dickson2_seq
)

In [ ]:
x_ = x[0,:] # not required to be 1D, just for example
# dickson with alpha=0 are monomials x^n, or use alpha=-1 for Fibonacci polynomials
plt.plot(x_, dickson1_seq([1,2,3,4,5], 0, x_).T)
plt.title('Dickson1')

In [ ]:
x_ = x[0,:] # not required to be 1D, just for example
# dickson with alpha=0 are monomials x^n, or use alpha=-1 for Fibonacci polynomials
plt.plot(x_, dickson2_seq([1,2,3,4,5], -1, x_).T)
plt.title('Dickson2')

## Qs

Qs are Greg Forbes' Q polynomials, $Q\text{bfs}$, $Q\text{con}$, and $Q_n^m$.  Qbfs and Qcon polynomials are radial only, and replace the 'standard' asphere equation.  The implementation of all three of these also uses a recurrence relation, although it is more complicated and outside the scope of this reference guide.  Each includes the leading prefix from the papers:

- $\rho^2(1-\rho^2)$ for $Q\text{bfs}$,
- $\rho^4$ for $Q\text{con}$,
- the same as $Q\text{bfs}$ for $Q_n^m$ when $m=0$ or $\rho^m \cos\left(m\theta\right)$ for $m\neq 0$

The $Q_n^m$ implementation departs from the papers in order to have a more Zernike-esque flavor.  Instead of having $a,b$ coefficients and $a$ map to $\cos$ and $b$ to $\sin$, this implementation uses the sign of $m$, with $\cos$ for $m>0$ and $\sin$ for $m<0$.

There are six essential functions:

In [ ]:
from prysm.polynomials import (
    Qbfs, Qbfs_seq,
    Qcon, Qcon_seq,
    Q2d, Q2d_seq,
)

In [ ]:
p = Qbfs(2,r)
p[mask]=np.nan
plt.imshow(p)

In [ ]:
p = Qcon(2,r)
p[mask]=np.nan
plt.imshow(p)

In [ ]:
p = Q2d(2, 2, r, t) # cosine term
p[mask]=np.nan
plt.imshow(p)

In [ ]:
p2 = Q2d(2, -2, r, t) # sine term
p2[mask]=np.nan
plt.imshow(p2)

In [ ]:
p2 = Q2d(2, -2, r, t) # sine term
p2[mask]=np.nan
plt.imshow(p2)

## XY

XY polynomials are implemented in the same manner as Code V.  A monoindexing scheme that is identical to Code V (**beginning from j=2**) is provided:

In [ ]:
from prysm.polynomials import (
    xy_polynomial,
    xy_polynomial_seq,
    j_to_xy
)

## Annular Domains

prysm does not have explicit implementations of annular polynomials, for example Mahajan's annular Zernikes.  Because all of the polynomial routines recursively generate the basis sets based on the input coordinates, modifications of the grid will produce versions of the polynomials that are orthogonal over the new domain, such as an annulus.  This underlying technique is actually how the radial basis of the Zernike polynomials is calculated.  The coordinates module features a `distort_annular_grid` function that performs this modification to a circle.  We will use it tho show Annular Zernikes.  We begin by making a circular aperture with huge central obscuration:

In [ ]:
eps = 0.5
maskod = circle(1, r)
maskid = circle(eps, r)
mask = maskod ^ maskid
plt.imshow(mask)
plt.title('Annular aperture')

Then we compute a distorted grid and call the polynomial routines as you would in any other case.  Note that the `norm` keyword argument uses analytic norms, which are not correct on distorted grids.  A helper function is provided by the polynomials module, `normalize_modes` to enforce normalizations on modes.

In [ ]:
from prysm.coordinates import distort_annular_grid
from prysm.polynomials import normalize_modes

In [ ]:
x, y = make_xy_grid(mask.shape, diameter=2)
r, t = cart_to_polar(x, y)

ran = distort_annular_grid(r, eps)

# nms = [noll_to_nm(j) for j in range(1,12)] # up to primary spherical
js = range(1,36)
nms = [fringe_to_nm(j) for j in js]

basis = zernike_nm_seq(nms, ran, t, norm=False)
basis = normalize_modes(basis, mask, to='std')
# basis_in_ap = basis[:,mask]
# print(basis_in_ap.shape)
# std_per_coef = basis_in_ap.std(axis=1)

# newaxis broadcasts (11,) -> (11,1,1) for numpy broadcast semantics
# basis = basis * (1/std_per_coef[:,np.newaxis,np.newaxis])
fig, axs = plt.subplots(ncols=4, figsize=(12,3))
for ax, i, name in zip(axs, (3,4,6,8), ('Power', 'Astigmatism', 'Coma', 'Spherical Aberration')):
    mode = basis[i].copy()
    mode[~mask]=np.nan
    ax.imshow(mode, cmap='RdBu')
    ax.set(title=name)

We can compare these distorted modes to ordinary Zernike polynomials:

In [ ]:
# note: r instead of ran, the undistorted grid
basis2 = zernike_nm_seq(nms, r, t, norm=False)
fig, axs = plt.subplots(ncols=4, figsize=(12,3))
for ax, i, name in zip(axs, (3,4,6,8), ('Power', 'Astigmatism', 'Coma', 'Spherical Aberration')):
    mode = basis2[i].copy()
    mode[~mask]=np.nan
    ax.imshow(mode, cmap='RdBu')

The `lstsq` function ignores data points marked as NaN, the variable `basis` from the block containing `eps` would be used in a least squares fit as per usual.  This method is compatible with all polynomial basis and is not limited to Zernikes.  Grid distortions that turn other shapes into a unit domain are similarly compatible, but they are not implemented in prysm at this time.

## Arbitrary Domains

The grid distortion trick provided out-of-the-box for annular apertures is very easy to implement and use for an annulus, and similarly easy for an ellipse.  More complex aperture shapes such as hexagons are less straightforward to derive a grid distortion for.  For these use cases, or to provide an alternative orthogonalization approach for annular apertures, prysm features a QR factorization based orthogonalization approach over an arbitrary aperture.  This is similar to performing a Gram-Schmidt process.  We'll repeat the annular example first:

In [ ]:
from prysm.polynomials import orthogonalize_modes

In [ ]:
basis3 = orthogonalize_modes(basis2, mask)
basis3 = normalize_modes(basis3, mask)

# purely cosmetic for plotting
nmask = ~mask
basis[:,nmask] = np.nan
basis2[:,nmask] = np.nan
basis3[:,nmask] = np.nan

fig, axs = plt.subplots(ncols=4, nrows=3, figsize=(12,9))
j = 0
for i, name in zip((3,4,6,8), ('Power', 'Astigmatism', 'Coma', 'Spherical Aberration')):
    raw_mode = basis2[i]
    grid_distorted_mode = basis[i]
    qr_mode = basis3[i]
    axs[0,j].imshow(raw_mode, cmap='RdBu')
    axs[1,j].imshow(grid_distorted_mode, cmap='RdBu')
    axs[2,j].imshow(qr_mode, cmap='RdBu')
    axs[0,j].set_title(name)
    j += 1

axs[0,0].set(ylabel='Circle Zernikes')
axs[1,0].set(ylabel='Annular Zernikes')
axs[2,0].set(ylabel='QR Orthogonalized Zernikes')

for ax in axs.ravel():
    ax.set_xticks([])
    ax.set_yticks([])

Note that both the basis generated by grid distortion and QR decomposition are orthogonal.  In the same way that there are many orthogonal bases included in prysm, these are just different orthogonal sets derived from Zernike polynomials (which are themselves derived from Jacobi polynomials).  We'll now show a second example, for a hexagonal aperture:

In [ ]:
from prysm.geometry import regular_polygon

In [ ]:
# whimsically rotated to show rarely used features
hexap = regular_polygon(6, 1, x, y, rotation=0)
im = plt.imshow(hexap, interpolation='bilinear')
plt.colorbar(im)

In [ ]:
basis2 = zernike_nm_seq(nms, r, t, norm=True)
basis3 = orthogonalize_modes(basis2, hexap)
basis3 = normalize_modes(basis3, hexap, to='std')

# this masking is cosmetic only for plotting!
basis3[:, ~hexap] = np.nan

fig, axs = plt.subplots(ncols=4, nrows=2, figsize=(12,6))
j = 0
cl = (-3,3)
for i, name in zip((3,6,10,28), ('Power', 'Coma', 'Trefoil', 'Primary Quadrafoil')):
    raw_mode = basis2[i]
    raw_mode[~hexap] = np.nan
    qr_mode = basis3[i]
    axs[0,j].imshow(raw_mode, cmap='RdBu', clim=cl)
    axs[1,j].imshow(qr_mode, cmap='RdBu', clim=cl)
    axs[0,j].set_title(name)
    j += 1

axs[0,0].set(ylabel='Circle Zernikes')
axs[1,0].set(ylabel='QR Orthogonalized Zernikes')

for ax in axs.ravel():
    ax.set_xticks([])
    ax.set_yticks([])

The similarity of the power, coma, and trefoil modes and dissimilarity of the higher order mode highlight a property of all polynomials: they are extremely similar, and largely irrespective of the domain for lower order modes.  Higher order modes will distort significantly to match any given domain.  If you are largely interested in lower order behaviors, orthogonality will likely not matter to you.  It is only when concerned with higher order modes that orthogonality will be of significance.

An additional property of using QR factorization, Gram-Schmidt, SVD, or other processes to produce orthogonal bases is that the output mode shapes depends on every detail of the inputs.  If the input basis changes, for example expanding Z1-Z11 in one case and Z1-Z36 in another, the output changes.  Similarly, the normalization radius of the input Zernikes (if those are used) must be specified consistently, as well as the exact centering between the polynomials and the aperture.  When the grid distortion techniques shown previously for annular apertures are used, the only relevant one of these effects is grid centering, which only impacts orthogonality and not mode shapes.